In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock
from langchain.chains import LLMChain
from langchain_community.document_loaders import PyPDFLoader
import boto3
import botocore

config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 3}
)

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)
path = (r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\2-Cloumn page\New_Fac_101.pdf")
loader = PyPDFLoader(path)
docs = loader.load()

template = """
Extract all the text datas in the given text documents and print all the text as a result in the formatted way as per the given document

Instruction:
- The given text document consists of two column,so if there is any word break inbetween
            for example: if the document consists like 'ava ilable' you must avoid the avoid the gap and print like 'available'
    like that for all the words.

{datas}
"""
qa_prompt = PromptTemplate(template=template, input_variables=["datas"])
# chain_type_kwargs = {"prompt": qa_prompt, "verbose": False}
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 20000})
llm_chain = LLMChain(prompt=qa_prompt, llm=llm, verbose= False)
result = llm_chain.run(datas= docs)

c:\Users\Lenovo\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [2]:
from fpdf import FPDF

def text_to_pdf(text, pdf_name='output2.pdf'):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    encoded_text = text.encode('latin-1', 'replace').decode('latin-1')
    
    pdf.multi_cell(0, 10, txt=encoded_text)
    pdf.output(pdf_name)

text_to_pdf(result)